## Data Preparation

In [15]:
import pandas as pd
import numpy as np

admissions = pd.read_csv('student_data.csv')

# Create dummy variables for the 'rank' feature
data = pd.concat([admissions, pd.get_dummies(admissions['rank'], prefix = 'rank')], axis = 1)
data = data.drop('rank', axis = 1)

# Standardize the 'gre' and 'gpa' features
for field in ['gre', 'gpa']:
    mean, std = data[field].mean(), data[field].std()
    data.loc[:,field] = (data[field] - mean) / std
    
#Split off random 10% of the data for testing
np.random.seed(21)
sample = np.random.choice(data.index, size = int(len(data) * 0.9), replace = False)
train_data, test_data = data.iloc[sample], data.drop(sample)

#Split into features and targets
features, targets = data.drop('admit', axis = 1), data['admit']
features_test, targets_test = test_data.drop('admit', axis = 1), test_data['admit']
                          

## Backpropagation

In [26]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Hyperparameters
n_hidden = 2 # Number of hidden nodes
epochs = 900
learn_rate = 0.005

n_records, n_features = features.shape
last_loss = None

# Initialize Weights
weights_input_hidden = np.random.normal(scale = n_features**-0.5, size = (n_features, n_hidden))
weights_hidden_output = np.random.normal(scale = n_features**-0.5, size = n_hidden)

for e in range(epochs):
    delta_w_input_hidden = np.zeros(weights_input_hidden.shape)
    delta_w_hidden_output = np.zeros(weights_hidden_output.shape)
    for x,y in zip(features.values, targets):
        ##Forward Pass##
        hidden_input = np.dot(x, weights_input_hidden)
        hidden_output = sigmoid(hidden_input)
        output = sigmoid(np.dot(hidden_output, weights_hidden_output))
        
        ##Backward Pass##
        # Neural network's prediction error
        error = (y - output)
        
        # Error term for the output node
        output_error_term = error * output * (1 - output)
        
        ## Propagate errors to the hidden layer ##
        # Calculate the hidden layer's contribution to the error
        hidden_error = np.dot(output_error_term, weights_hidden_output)
        
        # Calculate the hidden error term
        hidden_error_term = hidden_error * hidden_output * (1 - hidden_output)
        
        # Update the change in weights
        delta_w_hidden_output += output_error_term * hidden_output
        delta_w_input_hidden += hidden_error_term * x[:, None]
        
    # Update the weights
    weights_input_hidden += (learn_rate * delta_w_input_hidden) / n_records
    weights_hidden_output += (learn_rate * delta_w_hidden_output) / n_records
    
     # Printing out the mean square error on the training set
    if e % (epochs / 10) == 0:
        hidden_output = sigmoid(np.dot(x, weights_input_hidden))
        out = sigmoid(np.dot(hidden_output,
                             weights_hidden_output))
        loss = np.mean((out - targets) ** 2)

        if last_loss and last_loss < loss:
            print("Train loss: ", loss, "  WARNING - Loss Increasing")
        else:
            print("Train loss: ", loss)
        last_loss = loss

# Calculate accuracy on test data
hidden = sigmoid(np.dot(features_test, weights_input_hidden))
out = sigmoid(np.dot(hidden, weights_hidden_output))
predictions = out > 0.5
accuracy = np.mean(predictions == targets_test)
print("Prediction accuracy: {:.3f}".format(accuracy))
        

Train loss:  0.24372859910856
Train loss:  0.24314031034270805
Train loss:  0.24256829386141665
Train loss:  0.24201208909217223
Train loss:  0.24147124615323748
Train loss:  0.24094532587452736
Train loss:  0.2404338997886882
Train loss:  0.2399365500952694
Train loss:  0.2394528696006447
Train loss:  0.2389824616362671
Prediction accuracy: 0.650
